<a href="https://colab.research.google.com/github/dshan4585/AI_Expert_Lecture_Files/blob/main/Lab2_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 토이 텍스트 데이터 준비 (char-level)
text = "hello pytorch. Samsung AI expert"
chars = sorted(set(text))
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for ch, i in char2idx.items()}
seq = [char2idx[c] for c in text]

# 입력(x)과 목표(y) 시퀀스 생성
input_seq = seq[:-1]    # h e l l o   p y t o r t ...
target_seq = seq[1:]    # e l l o   p y t o r t c h ...

# 텐서 변환
x = torch.tensor(input_seq).unsqueeze(1)     # (seq_len, batch=1)
y = torch.tensor(target_seq)

# 원-핫 인코딩 함수
def one_hot(idx, num_classes):
    return torch.eye(num_classes)[idx]

x_onehot = one_hot(x, len(chars))  # (seq_len, 1, input_size)


In [ ]:

# 2. RNN 모델 정의
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=False)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h):
        out, h = self.rnn(x, h)
        out = self.fc(out.squeeze(1))  # (seq_len, output_size)
        return out, h





In [ ]:
hidden_size = 32
model = CharRNN(len(chars), hidden_size, len(chars))

# 3. 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 1
for epoch in range(1, epochs+1):
    h = torch.zeros(1, 1, hidden_size)  # 초기 은닉 상태
    optimizer.zero_grad()
    output, _ = model(x_onehot, h)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:

# 4. 텍스트 생성
model.eval()
with torch.no_grad():
    h = torch.zeros(1, 1, hidden_size)
    input_char = x_onehot[0].unsqueeze(0)  # 첫 글자
    generated = [idx2char[input_seq[0]]]
    for _ in range(len(text)-1):
        out, h = model(input_char, h)
        prob = torch.softmax(out, dim=1)
        idx = prob.argmax(dim=1).item()
        generated.append(idx2char[idx])
        print("idx2char[idx]:", idx2char[idx])
        # 다음 입력 준비
        input_char = one_hot(torch.tensor([[idx]]), len(chars))
    print("Generated:", ''.join(generated))